# Unsupervised Learning

## Background
This is intended to introduce you to classical approaches to unsupervised learning, using a data set that is famous in the face recognition literature.  To avoid making the class entirely dependent on scikit-learn's _fabulous_ implementations of clustering algorithms, you'll both code up your own implementation of the classical k-means algorithm, and compare it against some of sklearn's stuff.

If you haven't already done so, consider bookmarking some key resources:
+ The scikit-learn tutorial: https://scikit-learn.org/stable/tutorial/index.html , especially the unsupervised learning section (https://scikit-learn.org/stable/tutorial/statistical_inference/unsupervised_learning.html)
+ Scikit-learn's cluster analysis guide (https://scikit-learn.org/stable/modules/clustering.html). Focus on sections 2.3.1, 2.3.2, 2.3.6.{1,2}, 2.3.7, and 2.3.11.  it's a good idea to commit the sklearn approach to inference to your memory. Most of the clustering methods work by creating an object and then `fit()`ing it to some data.  This makes it easy to unplug one clustering approach and plug in another, which can be useful when you're doing exploratory data analysis.

In [ ]:
import pickle
import os
import sys
import tqdm.notebook as tqdm
import numpy as np
import scipy
%matplotlib inline
import matplotlib.pyplot as plt
import random
import sklearn.datasets
import sklearn.cluster
import sklearn.metrics

import gdown
from sklearn.cluster import DBSCAN
from sklearn.metrics import davies_bouldin_score


## Intro
To prepare for this assignment, please visit and bookmark some resources that will help you get up to speed on scikit-learn's features:

+ The scikit-learn tutorial: https://scikit-learn.org/stable/tutorial/index.html
+ The unsupervised learning section (https://scikit-learn.org/stable/tutorial/statistical_inference/unsupervised_learning.html)
+ Scikit-learn's cluster analysis guide (https://scikit-learn.org/stable/modules/clustering.html). Focus on sections 2.3.1, 2.3.2, 2.3.6.{1,2}, 2.3.7, and 2.3.11.

It's a good idea to commit the sklearn approach to inference to your memory. Most of the clustering methods work by creating an object and then `.fit()`ing it to some data.  This makes it easy to unplug one clustering approach and plug in another, which can be useful when you're doing exploratory data analysis.

## LFW data

a. Using `sklearn.datasets.fetch_lfw_people()`, get a subset of the Labeled Faces in the Wild (LFW) dataset.
 + Specify `min_faces_per_person=50` to get several classes (identities), each with 50 or more images.
 + Specify `return_X_y=True`, and assign the output of `.fetch_lfw_people()` to the tuple `lfw_pattern,lfw_label`.  Do not use different variable names, otherwie some helper code below will fail.

After this is done, you'll have:

 + a variable `lfw_pattern`, which is a 1560 x 2914 pattern matrix (1560 patterns, 2914 features) stored in a numpy array of shape `(1560,2914)`, and
 + a variable `lfw_label` which is a numpy vector of shape `(1560,)` (which means you can only use one subscript to index it -- it's not 1560x1, it's 1560. Yes, this is weird; this is numpy.

This step will take a bit of time to complete the first time you do it with a fresh runtime. Subsequent fetches are faster because the data is cached in your runtime.

b. Write a bit of code to compute and print out the number of classes in this data set.

In [ ]:
#a.
lfw_pattern, lfw_label = sklearn.datasets.fetch_lfw_people(min_faces_per_person=50, return_X_y=True)
print('Pattern:')
print(lfw_pattern, '\n')

print('Label:')
print(lfw_label, '\n')

#b.
unique_classes = np.unique(lfw_label)
num_classes = len(unique_classes)

print("Number of classes in the dataset: ", num_classes)

## Visualize

The patterns in this data set are actually images.  Each feature in the pattern is the normalized intensity of a pixel, and the pattern is a 62x47 array of pixels "flattened" into a 2914-element vector.

Write a function `show_face(x,s)` which uses Matplotlib's `plt.imshow()` routine to render `x`, which will be one of the patterns (rows) in the LFW pattern matrix you just fetched, as a grayscale image.  Use `plt.title(s)` to add a title.

Test your function by running the second code cell below. It picks an image randomly from each class and calls your code to display it.

Notes:
 + you must render the image with a *grayscale* colormap so the face looks like a proper photograph. No strange colors or "negative" images! Look at the `cmap` option to `plt.imshow()` to accomplish this.
 + you will need to `.reshape()` the pattern supplied as `x` into a 62x47 Numpy array that you then supply to `.imshow()`.

In [ ]:
def show_face(x,s):

  # Reshape pattern
  image_data = x.reshape(62,47)

  # Set grayscale
  plt.imshow(image_data, cmap='gray')

  # Set title
  plt.title(s)

In [ ]:
for l in set(lfw_label):
    faces = [x for i,x in enumerate(lfw_pattern) if lfw_label[i] == l]
    face = random.choice(faces)
    show_face(face,f'Class {l}')
    plt.show()

# $k$-means

In the cells below, write a function `my_k_means(pattern,k,max_iterations=10,dist=scipy.spatial.distance.euclidean)` that implements the famous $k$-means clustering algorithm.  The parameters to the function are:
 + the pattern matrix `pattern` with rows as patterns and columns as features.
 + `k` -- the desired maximum number of clusters to return.
 + `max_iterations` -- the maximum number of iterations of the $k$-means loop that will occur before the algorithm stops trying to improve the clustering results. This is a keyword argument with a default value of 10.
 + `dist` -- the name of a function that will compute the distance between two patterns supplied as arguments. This is a keyword argument with a default value of `scipy.spatial.distance.euclidean` (which, despite the oh-so-fancy name, just computes Euclidean distance).

The output of the algorithm is a tuple of two items:
 + a label vector (use a numpy vector for this), that is the same length as the number of patterns, with each element being an integer between 0 and $k-1$, inclusive. Note: the algorithm may not use every label in its output (_i.e._, you could specify $k=4$ and get a result where only labels 0, 2, and 3 are used), and
 + The **sum of squared distances** (SSD) between the patterns and their assigned cluster center.

Your code should follow the following basic outline, which is provided in the code cell below:

 + Choose $k$ cluster centers at random from the data (Hint: `random.choice()`
 + Compute "current" cluster assignments -- label each pattern with the index of its closest cluster center:
  + use the `dist` function to calculate the distances between a pattern `x` and each of the cluster centers.
  + the index of the closest cluster center for `x` is the initial label for `x`. Hint: `np.argmin()`
 + Loop no more than `max_iterations` times
  + Update cluster centers -- the new cluster center for cluster $i$ is the centroid (_i.e._, mean or average) of all the patterns assigned to it. Hint: `np.mean()`
  + Compute "new" cluster assignments (labels) and compare them to the "current" ones
    + if no labels changed, break out of the loop
    + if one or more labels changed, make "current" assignments equal to "new" assignments
 + When loop ends, calculate SSD by adding up the squared distances between all patterns and their nearest cluster center.
 + return cluster assignments and SSD

Put all your $k$-means code in the code cell below. Your are welcome (and encouraged!) to define helper functions, especially if they make the "core" of the $k$-means algorithm more compact and readable. Hint: As specified above, you will perform pattern assignment in two places. Maybe the same function can be called twice!

In [ ]:
def my_k_means(pattern,k,max_iterations=10,dist=scipy.spatial.distance.euclidean):
  # Choose starting cluster centers at random
  indices = np.random.permutation(pattern.shape[0])
  centers = pattern[indices[:k], :]

  # Perform initial cluster membership assignments
  labels = np.argmin([np.apply_along_axis(dist, 1, pattern, center) for center in centers], axis=0)

  # Loop no more than max_iterations
  for _ in range(max_iterations):
    #  Update cluster centers using all pattern assignments
    centers = np.array([pattern[labels == i].mean(axis=0) for i in range(k)])

    #  Calculate new assgnments
    new_labels = np.argmin([np.apply_along_axis(dist, 1, pattern, center) for center in centers], axis=0)

    if np.all(labels == new_labels):
      break
    else:
      labels = new_labels

  # Calculate SSD
  ssd = sum(dist(pattern[i], centers[labels[i]])**2 for i in range(pattern.shape[0]))

  # Return final set of assignments and SSD
  return (labels, ssd)

In [ ]:
pattern,label = sklearn.datasets.make_blobs(n_samples=100,centers=2, n_features=2)

fig,(ax1,ax2) = plt.subplots(1,2)
ax1.set_title('Ground Truth labeling')
ax2.set_title('k-means labeling')
_ = ax1.scatter(pattern[:,0],pattern[:,1],c=label)

(clabel,ssd) = my_k_means(pattern,k=2)
print(f'GT labels:      {label}')
print(f'k-means labels: {clabel}')

_ = ax2.scatter(pattern[:,0],pattern[:,1],c=clabel)

# Clustering evaluation

Apply your $k$-means implementation  to the LFW data.  Your goal is a clustering that minimizes SSD. Since $k$-means picks its initial cluster centers at random, different runs with the same input data can end up with different results! Generally, $k$-means is run multiple times and the resulting labeling with the lowest SSD is the winner.

Search for values of $k$ between 8 and 20. For each value of $k$, run $k$-means ten times with `max_iterations` set to 100. Note: my implementation of $k$-means, not particularly optimized, took 20 minutes to run these 13 different values of $k$. Plot the SSD for each run at each value of $k$ as a dot on a single plot. On the same plot, provide a line plot of the minimum SSD.

It is *extremely* likely that your plot of minimum SSD versus k will show a consistent drop as k increases. In the text box below your code and the plot you generate, discuss why this is so. Hint: what would the SSD be if each pattern was in its own cluster (_i.e._, $k$ = the number of patterns)?

In [ ]:
# Arrays to store SSD values
ssd_vals = []
ssd_vals_min = []

# Range of k values to test
k_values = range(8, 21)

# Define a list of colors
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

# For each k value...
for i, k in enumerate(k_values):
    # Initialize list to store SSDs for this k
    k_ssds = []

    # Run k-means 10 times
    for _ in range(10):
        _, ssd = my_k_means(lfw_pattern, k=k, max_iterations=100)
        k_ssds.append(ssd)

    # Add SSDs for this k to master list
    ssd_vals.extend(k_ssds)

    # Calculate and store minimum SSD for this k
    ssd_vals_min.append(min(k_ssds))

    # Create scatter plot of SSDs with color corresponding to k
    plt.scatter([k]*10, k_ssds, color=colors[i % len(colors)])


# Create line plot of minimum SSDs
plt.plot(k_values, ssd_vals_min, label='Minimum SSD', color='red')

# Add labels and legend
plt.xlabel('K value')
plt.ylabel('SSD value')
plt.title("Homemade, slow, painful model")
plt.legend()

# Show the plot
plt.show()

# Sklearn

Read the documentation for sklearn's implementation of $k$-means [here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).
Repeat the previous experiment but use `sklearn.cluster.KMeans` instead of your own code to compute the k-means clustering. To maximize the apples-to-apples comparison, specify these parameters (in addition to a value of $k$) when you create the `KMeans` object:
 + `init='random'`
 + `n_init=1`
 + `max_iter=100`, and
 + `tol=1.0e-30`.

Note: with `n_init=1`, only one run of $k$-means is performed each time you call it - so you still need to have an inner loop that runs $k$-means 10 times.

Another note: This should run much faster than your home-brewed version of $k$-means.

Compute and show the same graph you computed/showed in Problem 4. Read the documentation to figure out how to get the output labels and the SSD from the `KMeans` object.

 In the text box below the code box, comment on anything you observed with regard to results, performance (run time), _etc_.

In [ ]:
# Arrays to store SSD values
sk_ssd_vals = []
sk_ssd_vals_min = []

# Range of k values to test
k_values = range(8, 21)

# Define a list of colors
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

# For each k value...
for i, k in enumerate(k_values):
    # Initialize list to store SSDs for this k
    k_ssds = []

    # Run k-means 10 times
    for _ in range(10):
        model = sklearn.cluster.KMeans(n_clusters=k, init='random', n_init=1, max_iter=100, tol=1.0e-30)
        model.fit(lfw_pattern)
        k_ssds.append(model.inertia_)

    # Add SSDs for this k to master list
    sk_ssd_vals.extend(k_ssds)

    # Calculate and store minimum SSD for this k
    sk_ssd_vals_min.append(min(k_ssds))

    # Create scatter plot    of SSDs with color corresponding to k
    plt.scatter([k]*10, k_ssds, color=colors[i % len(colors)])


# Create line plot of minimum SSDs
plt.plot(k_values, sk_ssd_vals_min, label='Minimum SSD', color='red')

# Add labels and legend
plt.xlabel('K value')
plt.ylabel('SSD value')
plt.title('Sklearn KMeans Model')
plt.legend()

# Show the plot
plt.show()

# Clustering data with no labels

This is the typical use case for clustering - you just have data, and want to learn something about its structure. Cluster analysis will help you uncover groupings.

Clustering metrics are often used to discern the proper number of clusters.

This problem asks you to evaluate two clustering algorithms' outputs on a dataset using two different clustering metrics.

But first, you need data.

In [ ]:
lnks = ['1ZHHf0DrwFS7FF5qR4AhjBmSnb4EQmVK9','1ZIjxI6Cb0oh2Pm6t1RNIdyMxHIfo8bid', \
        '1ZJPzyn6VXOUYNMOPjrZxjaiPQu5atBQV','1ZJqX0_ZzcFsDKJjrdrCT-F3-gTL1TYO5', \
        '1ZKAMdRcjS86KGZVxnIn57_fYpXnC1Ulg','1ZKkLpPnfiEI5-wqDd9C4wRw5Us8B7HX8', \
        '1ZKx5QbCRUSzT0siI4NeMrK_MTq355gMN','1ZLRr57fSuddUcyVN-tSffFrI1EBAdbfv', \
        '1ZMOF5beH5x3yl8Q4jHu0zxEVFpM3Hkie','1ZMcXxWiGLOOkWdF9KUH6ZlyuQbNerehF']

def geturl(digit):
    """returns a gdown-compatible URL for one of ten shared files."""
    pfx = 'https://drive.google.com/uc?id='
    # guard code
    if type(digit) is str: digit = int(digit)
    digit = digit % 10
    return pfx + lnks[digit]

MY_DIGIT = '3'

url = geturl(MY_DIGIT)
#print(url)
if os.path.isfile('mydata.pkl'):
    os.remove('mydata.pkl')
if not os.path.exists('mydata.pkl'):
    gdown.download(url,'mydata.pkl')
print('mydata.pkl is available in the runtime.')

In [ ]:
# Load the data
fp = open('mydata.pkl', 'rb')
pattern_matrix = pickle.load(fp)
print("Pattern Matrix:")
print(pattern_matrix.shape)

There are between two and 15 clusters in your data. Use k-means to investigate this.

Run k-means for the range of `k` values suggested above (2 to 15 clusters). Look at the output of the homogeneity and Davies-Bouldin scores (make sure you know what "better" values are for both scores), and see if one or more values of `k` suggest themselves as potential good values for the true number of clusters. In the text cell below your code cell, explain your reasoning. Note: this problem will not be graded on whether you get the correct value - it will be based on the quality of your argument.
+ Documentation on Davies-Bouldin score computation in sklearn [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html).
+ Documentation on homogeneity score computation in sklearn [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html).


In [ ]:
# Range of k values to test
k_values = range(2, 16)

# Initialize list to store Davies-Bouldin scores for each k
db_scores = []

# For each k value...
for k in k_values:
    # Run k-means
    kmeans = sklearn.cluster.KMeans(n_clusters=k, n_init=1, random_state=0).fit(pattern_matrix)

    # Compute Davies-Bouldin score
    db_score = sklearn.metrics.davies_bouldin_score(pattern_matrix, kmeans.labels_)

    # Add score to list
    db_scores.append(db_score)

# Create line plot of Davies-Bouldin scores
plt.plot(k_values, db_scores, marker='o')

# Add labels
plt.xlabel('k')
plt.ylabel('Davies-Bouldin Score')
plt.title('Davies-Bouldin Score for different k values')

# Show the plot
plt.show()

Repeat question 5(b), but use the DBSCAN clustering algorithm insead of $k$-means (documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)).
 + You'll need to play with the `eps` parameter to find a good value for the number of clusters. Values for `eps` between 0.5 and 2.0 are realistic.
 + If you get **any* -1 values in the output `labels_` attribute of the `DBSCAN` object, ignore that result and increase `eps`.

For each run of DBSCAN which does not yield any -1 labels, calculate the silhouette and Davies-Bouldin measures. See if one or more values of k suggest themselves as potential good values for the true number of clusters. Explain your reasoning in the text cel below the code cell. Note: this problem will not be graded on whether you get the correct value - it will be based on the quality of your argument.

In [ ]:
# Range of k values to test
eps_values = np.arange(0.5,2.0, 0.01)

# Initialize list to store Davies-Bouldin scores for each k
db_scores = []
valid_eps = []

# For each eps value...
for eps in eps_values:
    # Run DBSCAN
    dbscan = DBSCAN(eps=eps).fit(pattern_matrix)

    # Check if any labels are -1 (indicating noise)
    if -1 in dbscan.labels_:
        continue

    # Check if only one cluster is found
    if len(set(dbscan.labels_)) <= 1:
        continue

    # Compute Davies-Bouldin score
    db_score = davies_bouldin_score(pattern_matrix, dbscan.labels_)

    # Add score to list
    db_scores.append(db_score)
    valid_eps.append(eps)
    print(eps, db_score)

# Create line plot of Davies-Bouldin scores
plt.plot(valid_eps, db_scores, marker='o')

# Add labels
plt.xlabel('eps')
plt.ylabel('Davies-Bouldin Score')
plt.title('Davies-Bouldin Score for different eps values')

# Show the plot
plt.show()